This notebook assigns weights to the edges of the Donegal road graph using the coordinates of townlands found from querying the OSM API. The graph and townland population data must first be loaded from disk.

In [25]:
import config
import osmnx as ox
import networkx as nx
import pandas as pd
import graph.helpers.graph_helper as helper

In [26]:
graph_file = f"{config.graphml_path}/donegal_osm_simplified.graphml"
G = ox.load_graphml(graph_file)
population_data = pd.read_csv(f"{config.population_data_path}/donegal_townlands_all_coordinates.csv")
population_data.head()

,Unnamed: 0,townland,town,population,lat,lng
0,0,Abbey Island,Ballyshannon Rural,10.0,54.510056,-8.195737
1,1,Abbeylands,Ballyshannon Rural,121.0,54.513005,-8.204606
2,2,Adderville,Straid,54.0,55.239112,-7.434771
3,3,Adderwal,Gleann Léithín,9.0,54.905821,-8.168399
4,4,Admiran,Stranorlar,364.0,54.811157,-7.765547


Next the closest edges to each townland's coordinates are found

In [27]:
# get nearest edges to coordinates, returns an ndarray
nearest_edges = ox.get_nearest_edges(G, population_data["lng"], population_data["lat"], method="balltree")



In [28]:
# convert ndarray to list
nearest_edges_list = nearest_edges.tolist()
# get start (u) and end (v) points of each edge
start_points = [item[0] for item in nearest_edges_list]
end_points = [item[1] for item in nearest_edges_list]

# add these point to the population dataset
population_data["start"] = start_points
population_data["end"] = end_points

In [29]:
population_data.head()

,Unnamed: 0,townland,town,population,lat,lng,start,end
0,0,Abbey Island,Ballyshannon Rural,10.0,54.510056,-8.195737,1947246217,1947246154
1,1,Abbeylands,Ballyshannon Rural,121.0,54.513005,-8.204606,1947245998,1947245965
2,2,Adderville,Straid,54.0,55.239112,-7.434771,2120765476,2912801448
3,3,Adderwal,Gleann Léithín,9.0,54.905821,-8.168399,1902079898,1902080067
4,4,Admiran,Stranorlar,364.0,54.811157,-7.765547,1329135900,1327944100


In [30]:
# add default weight 1 to all edges
nx.set_edge_attributes(G, 1, "weight")
# check total weight of graph
G.size(weight='weight')


6505.0

In [31]:
# apply weights based on population values for closest edge to townland coordinates
for index, row in population_data.iterrows():
    weight_before = G[row["start"]][row["end"]][0]['weight']
    print(f"weight of edge {row['start']} - {row['end']} before: {weight_before}")
    G[row["start"]][row["end"]][0]["weight"] = weight_before + row["population"]
    print(f"weight of edge {row['start']} - {row['end']} after: "
          f"{G[row['start']][row['end']][0]['weight']}")

weight of edge 1947246217 - 1947246154 before: 1
weight of edge 1947246217 - 1947246154 after: 11.0
weight of edge 1947245998 - 1947245965 before: 1
weight of edge 1947245998 - 1947245965 after: 122.0
weight of edge 2120765476 - 2912801448 before: 1
weight of edge 2120765476 - 2912801448 after: 55.0
weight of edge 1902079898 - 1902080067 before: 1
weight of edge 1902079898 - 1902080067 after: 10.0
weight of edge 1329135900 - 1327944100 before: 1
weight of edge 1329135900 - 1327944100 after: 365.0
weight of edge 1885442121 - 794807600 before: 1
weight of edge 1885442121 - 794807600 after: 16.0
weight of edge 2417655996 - 427999473 before: 1
weight of edge 2417655996 - 427999473 after: 8.0
weight of edge 1329142156 - 1914032114 before: 1
weight of edge 1329142156 - 1914032114 after: 29.0
weight of edge 427999446 - 7475145002 before: 1
weight of edge 427999446 - 7475145002 after: 45.0
weight of edge 345006774 - 529112566 before: 1
weight of edge 345006774 - 529112566 after: 5.0
weight of 

In [32]:
graph = list(G.edges(data=True))

G.size(weight='weight')

164425.0

In [33]:
ox.save_graphml(G, filepath=f"{config.graph_path}/graphml/donegal_osm_weights_applied.graphml")


